# SKEMA Experiment

Testing out SKEMA using a pet example from [this paper on a compartmental COVID-19 Model](https://www.sciencedirect.com/science/article/pii/S0960077921000424). `./model` contains the Python implementation of this model which was copied from [this Jataware repository](https://github.com/jataware/COVID-19_Projections). The original script reads from two JSON files but for this SKEMA experiment, they have been combined into one `options.py`.

This notebook is assumes you follow the installation instructions from the [TA1 December Demo Repo](https://github.com/ml4ai/ASKEM-TA1-SKEMA-Demo-2022-12).

In [15]:
import os
import json
from pprint import pprint
from automates.utils.fold import dictionary_to_gromet_json, del_nulls
from skema.text_reading.mention_linking.gromet_linker import align_and_link
from automates.utils.script_functions import process_file_system
from automates.program_analysis.JSON2GroMEt.json2gromet import json_to_gromet

MODEL_DIR = os.path.abspath("./model")
RESULTS_DIR = os.path.abspath("./demo_results")
SYSTEM_FILES_PATH=os.path.join(MODEL_DIR, "filepaths.txt")
GROMET_PATH = os.path.join(RESULTS_DIR, "test-auto.json")


os.makedirs(RESULTS_DIR, exist_ok=True)


Help on package automates.program_analysis.Py2GrFN in automates.program_analysis:

NAME
    automates.program_analysis.Py2GrFN

PACKAGE CONTENTS
    cast
    py2grfn

FILE
    /home/five/skema-demo/automates/automates/program_analysis/Py2GrFN/__init__.py




## Code2FN

In [16]:
COLLECTION_NAME = os.path.join(RESULTS_DIR, "test")


process_file_system(COLLECTION_NAME, MODEL_DIR, SYSTEM_FILES_PATH)

/home/five/askem-experiments/skema/model/main.py
found numpy
found scipy.integrate
found matplotlib.pyplot
found options
found options
Calling IdCollapsePass------------------------

Calling ContainerScopePass-------------------

Calling VariableVersionPass-------------------

Calling GrfnVarCreationPass-------------------

Calling GrfnAssignmentPass-------------------

Calling LambdaExpressionPass-------------------

Calling ToGrometPass-----------------------
/home/five/askem-experiments/skema/model/options.py
Calling IdCollapsePass------------------------

Calling ContainerScopePass-------------------

Calling VariableVersionPass-------------------

Calling GrfnVarCreationPass-------------------

Calling GrfnAssignmentPass-------------------

Calling LambdaExpressionPass-------------------

Calling ToGrometPass-----------------------


In [20]:
# DOESN't work
chime_module_collection = json_to_gromet('test--Gromet-FN-auto.json')
pprint(chime_module_collection)

FileNotFoundError: [Errno 2] No such file or directory: 'test--Gromet-FN-auto.json'

In [ ]:
# This also doesn't seem to work
from automates.program_analysis.Py2GrFN import py2grfn as gen_gromet

In [21]:
with open(COLLECTION_NAME+"--Gromet-FN-auto.json", 'r') as f:
    chime_module_collection = json.load(f)
    



## [SKIP] METAL

cannot use METAL because the following objects are not available. 
```python
COMMENTS_PATH = os.path.join(RESULTS_DIR, "test-comments.json")
EXTRACTIONS_PATH = os.path.join(RESULTS_DIR, "test-extractions.json")
EMBEDDINGS_PATH = os.path.join(RESULTS_DIR, "embeddings.kv")
```

## SKEMA Server and FN Graph Service

Swagger docs for the SKEMA server REST API: http://localhost:8080/api-docs/

In [ ]:
from IPython.display import IFrame
IFrame('http://localhost:8080/api-docs/', width=700, height=350)

In [ ]:
import json
from requests import get, put, post, delete
#from graphviz import Source
BASE_URL="http://localhost:8080"

Programmatically verify the service is running:

In [ ]:
get(f"{BASE_URL}/ping").text

We can view the **memgraph web interace**, in separate brower window, open: `http://localhost:3000`

Load the GroMEt JSON file `CHIME_SVIIvR_core--Gromet-FN-auto.json` into the database. The operation returns the ID of the model that was just inserted - we will use this ID to query the model later.

In [ ]:
from requests import post
with open('test--Gromet-FN-auto.json') as f:
    r = post(f"{BASE_URL}/models", json=json.loads(f.read()))
    if r.status_code == 200:
        MODEL_ID = r.json()
        print(f"Model ID: {MODEL_ID}")
    else:
        print(json.loads(r.request.body))

We then get the IDs of all the models in the database to double check if our model ID is among them.

In [ ]:
get(f"{BASE_URL}/models").json()

Run a query to collect the names of all of the _named_ `opi`s (Output Ports) in the model.

In [ ]:
get(f"{BASE_URL}/models/{MODEL_ID}/named_opis").json()

The API endpoint for **deleting** a model.

In [ ]:
delete(f"{BASE_URL}/models/{MODEL_ID}").text

Get the IDs of all the models in the database again to check that the model has been deleted.

In [ ]:
get(f"{BASE_URL}/models").json()